## qLDPC CSS-T Codes
Generating the corresponding generator matrices for the the binary codes of the $Q(C_1, C_2)$ code. Based on Theorem 9 of the paper *Toward Quantum CSS-T Codes from Sparce Matrices*

In [ ]:
from platform import python_version

print(python_version())

In [ ]:
import sympy as sp
import numpy as np
from itertools import combinations_with_replacement

In [ ]:
def schur(A: sp.Matrix, B: sp.Matrix) -> sp.Matrix:
    if A.shape != B.shape:
        raise ValueError("Matrices must have same shape.")
    return A.multiply_elementwise(B)

def square(A: sp.Matrix) -> sp.Matrix:
    m, n = A.shape
    C = sp.zeros((m * m, n))

    l = 0
    for i in range(m):
        for j in range(m):
            C[l, :] = schur(A.row(i), A.row(j))  
            l += 1
    return C

def QCLD(L: int, A: sp.Matrix) -> sp.Matrix:
    P = sp.zeros(L)
    for i in range(L - 1):
        P[i, i + 1] = 1
    P[L - 1, 0] = 1

    m, n = A.shape
    H = sp.zeros(m * L, n * L)
    for i in range(m):
        for j in range(n):
            exp = int(A[i, j]) % L
            if exp == -1:
                block = sp.zeros(L)
            else:
                block = (P ** exp) % 2
            H[i*L:(i+1)*L, j*L:(j+1)*L] = block

    return H % 2

In [ ]:
class LinearCode:
    def __init__(self, G: sp.Matrix):
        self.G = G.applyfunc(lambda x: int(x) % 2)
        self.G = self.gf2_rref(self.G)

    def __eq__(self, other):
        return self.G.equals(other.G)
    
    @staticmethod
    def gf2_rref(M):
        M = M.applyfunc(lambda x: int(x) & 1)
        rows, cols = M.shape
        r = 0
        for c in range(cols):
            pivot = next((i for i in range(r, rows) if M[i, c] == 1), None)
            if pivot is None:
                continue
            if pivot != r:
                M.row_swap(pivot, r)
            for i in range(rows):
                if i != r and M[i, c] == 1:
                    M.row_op(i, lambda v, j: (v + M[r, j]) % 2)
            r += 1
            if r == rows:
                break
        nonzero_rows = [M.row(i) for i in range(M.rows) if any(M.row(i))]
        return sp.Matrix(nonzero_rows)    
    
    def generator_matrix(self):
        return self.G
    
    def dimension(self):
        return self.G.rank()
        
    def dual(self):
        n = self.G.cols
        if self.G.rows == 0:
            return LinearCode(sp.eye(n))
        ns = self.G.nullspace()
        if not ns:
            return LinearCode(sp.zeros(0, n))
        ns_bin = [v.applyfunc(lambda x: int(x) & 1) for v in ns]
        ns_mat = sp.Matrix.hstack(*ns_bin).T % 2
        return LinearCode(ns_mat)
    
    def schur_square(self):
        if self.G.rows == 0:
            return LinearCode(sp.zeros(0, self.G.cols))
        rows = []
        for i, j in combinations_with_replacement(range(self.G.rows), 2):
            rows.append((self.G.row(i).multiply_elementwise(self.G.row(j))) % 2)
        return LinearCode(sp.Matrix.vstack(*rows)) if rows else LinearCode(sp.zeros(0, self.G.cols))

    def meet(self, other):
        if self.G.rows == 0 or other.G.rows == 0:
            return LinearCode(sp.zeros(0, self.G.cols))
        d1 = self.dual()
        d2 = other.dual()
        G_combined_dual = sp.Matrix.vstack(d1.G, d2.G) % 2
        combined_dual = LinearCode(G_combined_dual)
        return combined_dual.dual()

    def __repr__(self):
        return f"LinearCode({self.G.rows}x{self.G.cols}, dim={self.G.rows})"

In [ ]:
L = 4
A = sp.Matrix([
    [3, 1, 2, 1],
    [3, 3, 2, 3]
])

G = QCLD(L, A)
G_rref = LinearCode.gf2_rref(G)
print(G.rows, G_rref.rank())
# sp.pprint(G_rref)

In [ ]:
## Check meet function: adding a vector not in C1 to form C3, then C1 meet C3 should be C1
C1 = LinearCode(G)
k, n = G.shape
print("C1 dimension:", C1.dimension())
print("C1 length:", n)
H = C1.dual().generator_matrix() 
vec_not_in_C1 = sp.Matrix([[1,0,1,1,1,1,0,1,1,1,1,1,1,0,0,1]]).T
# print(vec_not_in_C1.shape)
# is_all_ones_in_C1 = H@vec_not_in_C1 % 2
# print("Is [1 1 ...] in C1?", is_all_ones_in_C1)
C3 = LinearCode(sp.Matrix.vstack(G, vec_not_in_C1.T))
print("C3 dimension (C1 + vector):", C3.dimension())

C1_new = C3.meet(C1)
print("C1_new dimension (C1 meet C3):", C1_new.dimension())
print("C1_new == C1?", C1_new == C1)


In [ ]:

C1 = LinearCode(G)
C2 = C1.meet(LinearCode(G).schur_square().dual())
C2_dual = C2.dual()
print("\nC1 dimension:", C1.dimension())
print("C2 dimension:", C2.dimension())
print("C2 dual dimension:", C2_dual.dimension())

# print("C1 generator matrix:")
# sp.pprint(C1.generator_matrix())
# print("\nC2 generator matrix:")
# sp.pprint(C2.generator_matrix())
# print("\nDual(C2) generator matrix:")
# sp.pprint(C2_dual.generator_matrix())

### Finding $H_X$ and $H_Z$ given $G_1$ and $G_2$
Attempts to generate the parity check matrix for a punctured CSS-T code using the binary geneator matrices. Note that the condtion $H_X * H_Z^T == 0$, which is how we verify our answer. We use the matrices provided in Example 17 of the paper.

Using RREF we rewrite the geneator matrix with dimensions $n$ x $k$ as $G = [I_k | P]$ where $P$ is a $k$ x $(n-k)$ matrix. Then the parity check matrix can be written as $H = [P^T | I_{n-k}]$. 

Matrix from Example 17: 

$$
G_1 =
\begin{bmatrix}
1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 \\
1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
1 & 1 & 1 & 1 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 1 & 0 & 0 & 0 & 0 \\
1 & 1 & 0 & 0 & 1 & 1 & 0 & 0 & 1 & 1 & 0 & 0 & 1 & 1 & 0 & 0 \\
1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0
\end{bmatrix}
$$

Matrix from Example 16:

$$
G_1 =
\begin{bmatrix}
1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 1 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 1 \\
0 & 0 & 0 & 0 & 1 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 1
\end{bmatrix}
$$

The code to generate the geneator matrices is in [`paritycheck.g`](paritycheck.g) made using GAP.